# Midterm 1 Assignment 6

#### Various Imports

In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

# Implementation of necessary methods

### Convolution Method

In [ ]:
def img_filter_convolution(image, filter, padtype=cv2.BORDER_REPLICATE):
    x_size, y_size = image.shape
    filter_size = filter.shape[0]
    padding = filter_size//2
    padded_image = cv2.copyMakeBorder(image, padding, padding, padding, padding, padtype)
    filtered_img = np.zeros((x_size, y_size))

    for x in range(x_size):
        for y in range(y_size):
            filtered_img[x, y] = (padded_image[x: x+filter_size, y:y+filter_size] * filter).sum()
    return filtered_img

### Gaussian Filter Implementation
Implemented just for testing purposes

In [ ]:
def gaussian_filter(scale, size=None):
    if size is None:
        size = int(2 * np.ceil(3*scale)) + 1
    v = np.arange((-size // 2) + 1, (size // 2) + 1)
    print(v)
    x = v * np.ones((size, size))
    y = x.T
    filter = 1/(2*np.pi*scale**2) * np.exp(-(x*x + y*y)/(2*scale**2))
    return filter

### LoG Filter Implementation

In [ ]:
def LoG(scale, size=None):
    if size is None:
        size = int(2 * np.ceil(3*scale)) + 1
    v = np.arange((-size // 2) + 1, (size // 2) + 1)
    x = v * np.ones((size, size))
    y = x.T
    gaussian = 1/(2*np.pi*scale**2) * np.exp(-(x*x + y*y)/(2*scale**2))
    log_filter = ((x*x + y*y)/(scale**4) - 2/(scale**2))*gaussian
    return log_filter


### Blob Detection Implementation

In [ ]:
def blob_detection(image, scale):
    filter = LoG(scale)
    output_image = img_filter_convolution(image, filter)
    return output_image

## Workflow for a single image

In [ ]:
image = cv2.imread("/home/davide/uni/ISPR-Midterms/Midterm1/MSRC_ObjCategImageDatabase_v1/1_11_s.bmp")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

plt.imshow(gray, cmap="gray")
plt.show()

Run blob detection on the image at a given scale

In [ ]:
scale = 10
tmp = blob_detection(gray, scale)
plt.imshow(tmp, cmap="gray")
plt.show()

Normalize values in the image between 0 and 255 to make it easier to threshold image later. <br>
Then, threshold image at a given threshold. This binary image will serve as the basis of blob detection.

In [ ]:
output = tmp * tmp
print(output.min(), output.max())
output = 255 * (output - output.min())/(output.max() - output.min())
ret, thresholded_output = cv2.threshold(output, 125, 255, cv2.THRESH_BINARY)
thresholded_output = thresholded_output.astype(np.uint8)

plt.imshow(thresholded_output, cmap="gray")
plt.show()

Each circle has a radius of scale * sqrt(2) to best cover the area of maximum response of the LoG. Simply drawing circles on points not zeroed out from the threshold results in a lot of overlapping circles. To better visualize blobs an approximation using contours can be used.

In [ ]:
fig, ax = plt.subplots()
ax.imshow(image)
x, y = output.shape
for row in range(x):
    for column in range(y):
        if thresholded_output[row, column] != 0:
            c = plt.Circle((column, row), scale*np.sqrt(2), color="red", linewidth = 0.5, fill=False)
            ax.add_patch(c)
ax.plot()
plt.show()

Find contours of the blobs appearing in the previously computed binary image obtained from thresholding.

In [ ]:
tmp_image = image.copy()
contours, hierarchies = cv2.findContours(thresholded_output, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
for i in contours:
    cv2.drawContours(tmp_image, [i], -1, (0, 255, 0), 2)
plt.imshow(tmp_image)

Compute the centers of the previously found contours. Method taken from https://www.geeksforgeeks.org/python-opencv-find-center-of-contour/

In [ ]:
centers = []
for i in contours:
    M = cv2.moments(i)
    if M['m00'] != 0:
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
        centers.append((cx, cy))
    print(f"x: {cx} y: {cy} scale: {scale}")

The computed centers are used as centers of the circles representing the found blobs. This is an approximation used to prune the otherwise large number of overlapping circles.

In [ ]:
fig, ax = plt.subplots()
ax.imshow(image)
for x, y in centers:
    c = plt.Circle((x, y), scale*np.sqrt(2), color="red", linewidth = 1.5, fill=False)
    ax.add_patch(c)
ax.plot()
plt.show()

# Image 1

In [ ]:
image = cv2.imread("/home/davide/uni/ISPR-Midterms/Midterm1/MSRC_ObjCategImageDatabase_v1/sunflowers.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

outputs = []
scales = [1, 5, 10, 20, 30]
for scale in scales:
    output = blob_detection(gray, scale)
    output = 255 * (output - output.min())/(output.max() - output.min())
    outputs.append((output * output, scale))


In [ ]:
centers = []
for output, scale in outputs:
    ret, thresholded_output = cv2.threshold(output, 150, 255, cv2.THRESH_BINARY)
    thresholded_output = thresholded_output.astype(np.uint8)
    contours = cv2.findContours(thresholded_output,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)[-2]
    for i in contours:
        M = cv2.moments(i)
        if M['m00'] != 0:
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            centers.append((cx, cy, scale))
        # print(f"x: {cx} y: {cy} scale: {scale}")

In [ ]:
fig, ax = plt.subplots()
ax.imshow(image)
for x, y, scale in centers:
    c = plt.Circle((x, y), scale*np.sqrt(2), color="red", linewidth = 1.5, fill=False)
    ax.add_patch(c)
# x, y = output.shape
# for row in range(x):
#     for column in range(y):
#         if thresholded_output[row, column] != 0:
#             c = plt.Circle((column, row), scale*np.sqrt(2), color="red", linewidth = 0.5, fill=False)
#             ax.add_patch(c)
ax.plot()
plt.show()